In [1]:
import keras
import h5py

Using Theano backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
import numpy as np
from PIL import Image
import argparse
import math
import random

In [3]:
import numpy as np
import os
from PIL import Image
def crop_center(img, crop_size=None):
    (y,x,_) = img.shape
    xo, yo = (x//2, y//2)
    crop_size = int(min(x,y)/2) if not crop_size else crop_size
    result = img[yo-crop_size:yo+crop_size,xo-crop_size:xo+crop_size]
    return result
directory = './sample/'
images = []
for imagefile in os.listdir(directory):
    image = Image.open(os.path.join(directory, imagefile), 'r')
    img_array = np.asarray(image, 'float32')
    images.append(crop_center(img_array))
images = np.array(images, dtype="float32")/255

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

#f, axarr = plt.subplots(4, 4, figsize=(10,10))
#for i in range(4):
    #for j in range(4):
        #plt.axes(axarr[i,j])
        #plt.imshow(images[4*i+j])
        #plt.xticks([])
        #plt.yticks([])
#plt.show()

In [5]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(4*64*64))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    # reshaped matrix should be 1/4 of output shape
    model.add(Reshape(( 64, 64,4), input_shape=(4*64*64,)))
    model.add(UpSampling2D(size=(2, 2), data_format='channels_last'))
    model.add(Conv2D(64, (5, 5), border_mode='same', data_format='channels_last'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2), data_format='channels_last'))
    model.add(Conv2D(3, (5, 5), border_mode='same', data_format='channels_last'))
    model.add(Activation('tanh'))
    return model

In [6]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, (5, 5),
                        border_mode='same',
                        input_shape=(256, 256,3), data_format='channels_last'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(Conv2D(128, (5, 5), border_mode='same', data_format='channels_last'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
def generator_containing_discriminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [8]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

In [9]:
def generate(BATCH_SIZE, nice=False, prefix=""):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator.h5py')
    if nice:
        discriminator = discriminator_model()
        discriminator.compile(loss='binary_crossentropy', optimizer="SGD")
        discriminator.load_weights('discriminator.h5py')
        noise = np.zeros((BATCH_SIZE*20, 100))
        for i in range(BATCH_SIZE*20):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        d_pret = discriminator.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE, 1) +
                               (generated_images.shape[2:]), dtype=np.float32)
        for i in range(int(BATCH_SIZE)):
            idx = int(pre_with_index[i][1])
            nice_images[i, 0, :, :] = generated_images[idx, 0, :, :]
        image = combine_images(nice_images)
    else:
        noise = np.zeros((BATCH_SIZE, 100))
        for i in range(BATCH_SIZE):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(prefix+
        "generated_image.png")
def example(prefix=""):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator.h5py')
    noise = np.zeros((BATCH_SIZE, 100))
    for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
    image = generator.predict(noise, verbose=1)
    print('generated image '+str(image.shape))
    image = image*127.5+127.5
    image = image[0,:,:,:]
    Image.fromarray(image.astype(np.uint8)).save(prefix+"generated_image.png")

In [10]:
losses_graph = []
def train(BATCH_SIZE,continue_save=True):
    import numpy as np
    import os
    from PIL import Image
    def crop_center(img):
        (y,x,_) = img.shape
        xo, yo = (x//2, y//2)
        crop_size = int(min(x,y)/2)
        result = img[yo-crop_size:yo+crop_size,xo-crop_size:xo+crop_size]
        return result
    directory = './small/'
    images = []
    for imagefile in os.listdir(directory):
        image = Image.open(os.path.join(directory, imagefile), 'r')
        img_array = np.asarray(image, 'float32')
        images.append(crop_center(img_array))
    images = np.array(images, dtype="float32")
    X_train = images
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    
    generator = generator_model()
    discriminator = discriminator_model()

    discriminator_on_generator = \
        generator_containing_discriminator(generator, discriminator)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    discriminator_on_generator.compile(
        loss='binary_crossentropy', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)
    noise = np.zeros((BATCH_SIZE, 100))
    
    
    if continue_save and os.path.isfile('generator.h5py') and os.path.isfile('discriminator.h5py'):
        generator.load_weights('generator.h5py')
        discriminator.load_weights('discriminator.h5py')
    for epoch in range(50):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)
                
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            
            d_loss = discriminator.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            discriminator.trainable = False
            
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * BATCH_SIZE)
            discriminator.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if index % 5 == 0:
                generator.save_weights('generator.h5py', True)
                discriminator.save_weights('discriminator.h5py', True)
            example(prefix="%d_"%(epoch*int(X_train.shape[0]/BATCH_SIZE)+index))
            print("weights saved for index %d" % (epoch*int(X_train.shape[0]/BATCH_SIZE)+index))
            losses_graph.append((d_loss, g_loss))
    return losses_graph

In [11]:
from PIL import Image

In [12]:
BATCH_SIZE = 32
losses_graph = train(BATCH_SIZE)
losses_graph

/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  app.launch_new_instance()
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), data_format="channels_last", padding="same")`
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), data_format="channels_last", padding="same")`
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), data_format="channels_last", input_shape=(256, 256,..., padding="same")`
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), data_format="channels_last", pa

Epoch is 0
Number of batches 4
batch 0 d_loss : 0.665695
batch 0 g_loss : 0.496203


/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  app.launch_new_instance()
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), data_format="channels_last", padding="same")`
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), data_format="channels_last", padding="same")`


32/32 [==============================] - 15s
generated image (32, 256, 256, 3)
weights saved for index 0
batch 1 d_loss : 0.580166
batch 1 g_loss : 0.384631
32/32 [==============================] - 15s
generated image (32, 256, 256, 3)
weights saved for index 1
batch 2 d_loss : 0.536770
batch 2 g_loss : 0.298484
32/32 [==============================] - 15s
generated image (32, 256, 256, 3)
weights saved for index 2
batch 3 d_loss : 0.524399
batch 3 g_loss : 0.227753
32/32 [==============================] - 14s
generated image (32, 256, 256, 3)
weights saved for index 3
Epoch is 1
Number of batches 4
batch 0 d_loss : 0.468935
batch 0 g_loss : 0.190340
32/32 [==============================] - 15s
generated image (32, 256, 256, 3)
weights saved for index 4
batch 1 d_loss : 0.482383
batch 1 g_loss : 0.166250
32/32 [==============================] - 15s
generated image (32, 256, 256, 3)
weights saved for index 5
batch 2 d_loss : 0.492619
batch 2 g_loss : 0.151214
32/32 [====================

KeyboardInterrupt: 

In [17]:
losses_graph

NameError: name 'losses_graph' is not defined

In [13]:
generate(BATCH_SIZE)

/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  app.launch_new_instance()
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), data_format="channels_last", padding="same")`
/home/rudolf/.local/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), data_format="channels_last", padding="same")`


KeyboardInterrupt: 

In [14]:
from keras.applications import vgg16
from keras import backend as K

In [15]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [16]:
img_nrows,img_ncols,f_outputs= 256,256,

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
if K.image_data_format() == 'channels_first':
    x = np.random.uniform(0, 255, (1, 3, img_nrows, img_ncols)) - 128.
else:
    x = np.random.uniform(0, 255, (1, img_nrows, img_ncols, 3)) - 128.

In [ ]:
# util function to open, resize and format pictures into appropriate tensors
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
import time
from scipy.misc import imsave
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))